In [139]:
'''
Title: Cryptocurrency - Technical Indicators
Author: Nicholas Ficeto
Description: I made this Jupyter Notebook to learn more about popular technical indicators in the stock world, and
how they can apply to cryptocurrencies.  This notebook was purely made for fun and learning purposes; whether 
technical indicators like bollinger bands are actually useful applied to cryptocurrencies remains a subject of 
debate.  Currently, I've added bollinger bands, but I'd like to add other features/indicators (RSI, MACD).
'''

'''
THINGS TO DO:
-Add charts for altcoins
-Add vertical lines when OHLC crosses a bollinger band
'''

'\nTitle: Cryptocurrency - Technical Indicators\nAuthor: Nicholas Ficeto\nDescription: I made this Jupyter Notebook to learn more about popular technical indicators in the stock world, and\nhow they can apply to cryptocurrencies.  This notebook was purely made for fun and learning purposes; whether \ntechnical indicators like bollinger bands are actually useful applied to cryptocurrencies remains a subject of \ndebate.\n'

In [206]:
import os
import numpy as np
import pandas as pd
import pickle
import quandl
from datetime import datetime
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
from shapely.geometry import LineString

In [129]:
'''
Get crypto data from Quandl.  Special "quandl_code" must be passed in (see Quandl website for formatting,
e.g. BCHARTS/KRAKENUSD, where BCHARTS returns Bitcoin data from the Kraken exchange in USD).
'''
def get_quandl_data(quandl_code):
    '''Create path name to which quandl data will be cached and pickled.'''
    cache_path = '{}.pkl'.format(quandl_code + str(datetime.now().date())).replace('/','-')
    '''First check if data for this Quandl code is already cached to avoid superfluous API calls.'''
    try:
        file = open(cache_path)
        df = pickle.load(file)
        print('Loaded cached file {}'.format(quandl_code))
    except(OSError, IOError):
        print('Downloading {} from Quandl'.format(quandl_code))
        df = quandl.get(quandl_code, start_date='2018-01-01', returns="pandas")
        df.to_pickle(cache_path)
        print('Cached {} in {}'.format(quandl_code, cache_path))
    return df

In [130]:
btc_prices_kraken = get_quandl_data('BCHARTS/KRAKENUSD')

Cached BCHARTS/KRAKENUSD in BCHARTS-KRAKENUSD2018-09-09.pkl


In [131]:
# Remove unwanted "0" values from dataframe
btc_prices_kraken.replace(0, np.nan, inplace=True)

In [132]:
'''
Add OHLC column in-place, whose values will be used to make technical indicators.
'''
def add_ohlc(df):
    df['OHLC'] = df[['Open','High','Low','Close']].mean(axis=1)

In [133]:
add_ohlc(btc_prices_kraken)

In [134]:
'''
Pass in a reference to a dataframe to add columns containing data to make bollinger bands in-place (no return df).
Dataframe must have an 'OHLC' column.
Code source/inspiration: http://www.pythonforfinance.net/2017/07/31/bollinger-band-trading-strategy-backtest-in-python/
'''
def create_boll_bands(df):
    #Set number of days and standard deviations to use for rolling lookback period for Bollinger band calculation
    window = 21
    no_of_std = 2

    #Calculate rolling mean and standard deviation using number of days set above
    rolling_mean = df['OHLC'].rolling(window).mean()
    rolling_std = df['OHLC'].rolling(window).std()

    #create two new DataFrame columns to hold values of upper and lower Bollinger bands
    df['Rolling Mean'] = rolling_mean
    df['Bollinger High'] = rolling_mean + (rolling_std * no_of_std)
    df['Bollinger Low'] = rolling_mean - (rolling_std * no_of_std)

In [135]:
'''
Create bollinger bands for the BTC prices dataframe.
'''
create_boll_bands(btc_prices_kraken)

In [248]:
'''
Convert pandas series to a list of tuples containing coordinates for the given line (column).
'''
def series_to_coordinates(df, column):
    coors = []
    for i in range(len(df)):
        x = btc_prices_kraken[column][i]
        y = i
        coors.append((x, y))
    return coors
'''
Find where the crypto trend line intersects with bollinger bands, and return a list of these points.
'''
def find_intersections(df):
    coors1 = series_to_coordinates(df, 'OHLC')
    coors2 = series_to_coordinates(df, 'Bollinger High')
    coors3 = series_to_coordinates(df, 'Bollinger Low')
    '''Cut off first 20 points where there is no data for Bollinger Bands'''
    coors1 = coors1[20:]
    coors2 = coors2[20:]
    coors3 = coors3[20:]
    l1 = LineString(coors1)
    l2 = LineString(coors2)
    l3 = LineString(coors3)
    intersection_high = l1.intersection(l2)
    intersection_low = l1.intersection(l3)
    inter_points_high = [list(p.coords)[0] for p in intersection_high]
    inter_points_low = [list(p.coords)[0] for p in intersection_low]
    return inter_points_high + inter_points_low
intersections = find_intersections(btc_prices_kraken)

In [275]:
intersections_trace = go.Scatter(
    x = [btc_prices_kraken.index[round(pair[1])] for pair in intersections],
    y = [pair[0] for pair in intersections],
    mode = 'markers',
    name = "Bollinger Intersections"
)

In [11]:
# Chart the BTC pricing data
btc_trace = go.Scatter(x=btc_prices_kraken.index, y=btc_prices_kraken['OHLC'], name='OHLC')
boll_high_trace = go.Scatter(x=btc_prices_kraken.index, y=btc_prices_kraken['Bollinger High'], name='Bollinger High')
boll_low_trace = go.Scatter(x=btc_prices_kraken.index, y=btc_prices_kraken['Bollinger Low'], name='Bollinger Low')
layout = dict(title = 'BTC Prices (Kraken Exchange)', xaxis=dict(title='Date'), yaxis=dict(title='Price (USD)'))
fig = dict(data=[btc_trace, boll_high_trace, boll_low_trace, intersections], layout=layout)
py.iplot(fig)

NameError: name 'btc_prices_kraken' is not defined